In [16]:
# Import the packages needed
import numpy as np, pandas as pd
import urllib
import json
import os

In [28]:
# Load the data as a JSON dict using the API

with urllib.request.urlopen("http://api.511.org/transit/lines?api_key=f5c9c70c-3e13-4e76-8481-d3ecc866ef3f&operator_id=SF&format=json") as url:
    data = json.load(url)

In [35]:
data[-12]

{'Id': '81X',
 'Name': 'CALTRAIN EXPRESS',
 'TransportMode': 'bus',
 'PublicCode': '81X',
 'SiriLineRef': '81X',
 'Montiored': True,
 'OperatorRef': 'SF'}

In [24]:
IDs = []

i = 0

for item in data:
    ID = data[i]['Id']
    IDs.append(ID)
    
    i += 1

IDs

['82X',
 'TBUS',
 'KT',
 'M',
 'NX',
 '3',
 '8AX',
 '38AX',
 '55',
 'L',
 '14R',
 'J',
 '714',
 '10',
 '12',
 '1',
 '1AX',
 '1BX',
 '31AX',
 '31BX',
 '38BX',
 '2',
 '5',
 '5R',
 '6',
 '7',
 '7X',
 '8',
 '8BX',
 '9',
 '9R',
 '14',
 '14X',
 '18',
 '19',
 '21',
 '38R',
 '39',
 '41',
 '22',
 '23',
 '24',
 '25',
 '27',
 '43',
 '28',
 '28R',
 '44',
 '29',
 '30',
 '30X',
 '31',
 '33',
 '35',
 '36',
 '37',
 '38',
 '45',
 '47',
 '48',
 '49',
 '52',
 '54',
 '56',
 '57',
 'PM',
 'PH',
 'C',
 '66',
 '67',
 '76X',
 '81X',
 '88',
 '90',
 '91',
 'K-OWL',
 'L-OWL',
 'M-OWL',
 'N-OWL',
 'T-OWL',
 'F',
 'N',
 'S']

In [47]:
lines_to_scrape = ['L-OWL',
 'M-OWL',
 'N-OWL',
 'T-OWL',
 'F',
 'N',
 'S']

i = 0

# Loop through each route in the API you want

for route in lines_to_scrape:

    with urllib.request.urlopen("http://api.511.org/transit/timetable?api_key=f5c9c70c-3e13-4e76-8481-d3ecc866ef3f&operator_id=SF&line_id="+lines_to_scrape[i]+"&format=json") as url:
        data = json.load(url)

    # Loops through each time table and for each trip id, prints the stop id and arrival time

    # First I will make a dataframe with all of the time tables and then think about splitting them by time table or just reassign the vars for that later

    lines, directions, timetables, trip_ids, stop_ids, arrival_times = [], [], [], [], [], []

    for timetable in data['Content']['TimetableFrame']:
        for servicejourney in timetable['vehicleJourneys']['ServiceJourney']:
            for call in servicejourney['calls']['Call']:

                # Grab the line
                line = timetable['Name'].split(':')
                lines.append(line[0])
                
                # Inbound or outbound
                directions.append(line[1])
                
                # What time table it is
                timetables.append(line[2])
                
                # What the trip id is
                trip = servicejourney['id']
                trip_ids.append(trip)
                
                # What the stop id is
                stop = call['ScheduledStopPointRef']['ref']
                stop_ids.append(stop)
                
                # Grabs the arrival time
                arrival = call['Arrival']['Time']
                arrival_times.append(arrival)
    
    tidy_df = pd.DataFrame({'ROUTE':pd.Series(lines),
                       'direction':pd.Series(directions),
                       'timetable':pd.Series(timetables),
                       'trip_id':pd.Series(trip_ids),
                       'stop_id':pd.Series(stop_ids),
                       'TIME':pd.to_datetime(arrival_times)})

    filePath = os.path.join(r'','route_' + lines_to_scrape[i] + '_timetables_data' + '.csv')
    os.path.isfile(filePath)
    tidy_df.to_csv(filePath, index=False)

    print("Completed scraping route " + lines_to_scrape[i])

    i += 1  

Completed scraping route L-OWL
Completed scraping route M-OWL
Completed scraping route N-OWL
Completed scraping route T-OWL
Completed scraping route F
Completed scraping route N
Completed scraping route S


In [41]:
data['Content']['TimetableFrame']['vehicleJourneys']

{'ServiceJourney': [{'id': '8572588',
   'SiriVehicleJourneyRef': '8572588',
   'JourneyPatternView': {'RouteRef': {'ref': '328731'},
    'DirectionRef': {'ref': 'IB'}},
   'calls': {'Call': [{'order': '1',
      'ScheduledStopPointRef': {'ref': '13164'},
      'Arrival': {'Time': '06:50:00', 'DaysOffset': '0'},
      'Departure': {'Time': '06:50:00', 'DaysOffset': '0'}},
     {'order': '2',
      'ScheduledStopPointRef': {'ref': '13136'},
      'Arrival': {'Time': '06:55:00', 'DaysOffset': '0'},
      'Departure': {'Time': '06:55:00', 'DaysOffset': '0'}},
     {'order': '3',
      'ScheduledStopPointRef': {'ref': '17619'},
      'Arrival': {'Time': '07:04:00', 'DaysOffset': '0'},
      'Departure': {'Time': '07:04:00', 'DaysOffset': '0'}}]}},
  {'id': '8572589',
   'SiriVehicleJourneyRef': '8572589',
   'JourneyPatternView': {'RouteRef': {'ref': '328731'},
    'DirectionRef': {'ref': 'IB'}},
   'calls': {'Call': [{'order': '1',
      'ScheduledStopPointRef': {'ref': '13164'},
      'A

In [43]:
with urllib.request.urlopen("http://api.511.org/transit/timetable?api_key=f5c9c70c-3e13-4e76-8481-d3ecc866ef3f&operator_id=SF&line_id="+lines_to_scrape[i]+"&format=json") as url:
        data = json.load(url)

    # Loops through each time table and for each trip id, prints the stop id and arrival time

    # First I will make a dataframe with all of the time tables and then think about splitting them by time table or just reassign the vars for that later

lines, directions, timetables, trip_ids, stop_ids, arrival_times = [], [], [], [], [], []

for timetable in data['Content']['TimetableFrame']:
    for servicejourney in timetable['vehicleJourneys']['ServiceJourney']:
        for call in servicejourney['calls']['Call']:

            # Grab the line
            line = timetable['Name'].split(':')
            lines.append(line[0])

            # Inbound or outbound
            directions.append(line[1])

            # What time table it is
            timetables.append(line[2])

            # What the trip id is
            trip = servicejourney['id']
            trip_ids.append(trip)

            # What the stop id is
            stop = call['ScheduledStopPointRef']['ref']
            stop_ids.append(stop)

            # Grabs the arrival time
            arrival = call['Arrival']['Time']
            arrival_times.append(arrival)
    
    tidy_df = pd.DataFrame({'ROUTE':pd.Series(lines),
                       'direction':pd.Series(directions),
                       'timetable':pd.Series(timetables),
                       'trip_id':pd.Series(trip_ids),
                       'stop_id':pd.Series(stop_ids),
                       'TIME':pd.to_datetime(arrival_times)})

tidy_df.head()

TypeError: string indices must be integers

In [27]:
data[1]

KeyError: 1